Parameters and helper functions

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, inchi
import warnings
import os

warnings.filterwarnings('ignore')

# ============================================================
# PARAMETERS - Modify these as needed
# ============================================================
# Input file paths
COMPOUND_FILE_PATH = "/Users/bowen/Desktop/mcid/database/kegg_compounds_demo_100.csv"
LIPID_FILE_PATH = "/Users/bowen/Desktop/mcid/database/lipid/lipid_maps.csv"

# Output parameters
DATASOURCE = "KEGG"  # Data source name
RXN_LEVEL = "0 Reaction"  # Reaction level
CPD_ID_INDEX = "K0"  # Compound ID index suffix (e.g., K0 -> M0000001K0, M0000002K0, ...)

# Output file path
OUTPUT_FILE_PATH = "/Users/bowen/Desktop/mcid/database/kegg_compounds_demo_100.parquet"

print("=" * 60)
print("PARAMETERS")
print("=" * 60)
print(f"Compound file: {COMPOUND_FILE_PATH}")
print(f"Lipid file: {LIPID_FILE_PATH}")
print(f"Datasource: {DATASOURCE}")
print(f"Rxn level: {RXN_LEVEL}")
print(f"Compound ID index: {CPD_ID_INDEX}")
print(f"Output file: {OUTPUT_FILE_PATH}")
print("=" * 60)



# ============================================================
# HELPER FUNCTIONS
# ============================================================

def detect_and_load_file(file_path):
    """Detect file format (CSV or Parquet) and load accordingly."""
    _, ext = os.path.splitext(file_path.lower())
    if ext == '.parquet':
        print(f"  Detected Parquet format: {file_path}")
        return pd.read_parquet(file_path)
    elif ext in ['.csv', '.tsv', '.txt']:
        print(f"  Detected CSV format: {file_path}")
        return pd.read_csv(file_path)
    else:
        try:
            df = pd.read_parquet(file_path)
            print(f"  Loaded as Parquet: {file_path}")
            return df
        except:
            df = pd.read_csv(file_path)
            print(f"  Loaded as CSV: {file_path}")
            return df

def get_inchikey_main_block(smiles):
    """Get the main block (first 14 characters) of InChIKey from SMILES."""
    if pd.isna(smiles) or smiles == '' or smiles is None:
        return None
    try:
        mol = Chem.MolFromSmiles(str(smiles))
        if mol is None:
            return None
        inchikey = inchi.MolToInchiKey(mol)
        if inchikey:
            return inchikey.split('-')[0]
        return None
    except Exception:
        return None

def get_exact_mass(smiles):
    """Get exact mass from SMILES."""
    if pd.isna(smiles) or smiles == '' or smiles is None:
        return None
    try:
        mol = Chem.MolFromSmiles(str(smiles))
        if mol is None:
            return None
        return Descriptors.ExactMolWt(mol)
    except Exception:
        return None

def is_organic(smiles):
    """Check if a molecule is organic based on SMARTS patterns."""
    if pd.isna(smiles) or smiles == '' or smiles is None:
        return False
    try:
        mol = Chem.MolFromSmiles(str(smiles))
        if mol is None:
            return False
        
        # Must contain carbon to be organic
        has_carbon = mol.HasSubstructMatch(Chem.MolFromSmarts('[#6]'))
        if not has_carbon:
            return False
        
        # Exclude simple inorganic carbon compounds
        co2_pattern = Chem.MolFromSmarts('[O]=[C]=[O]')
        co_pattern = Chem.MolFromSmarts('[#6]#[#8]')
        carbonate_pattern = Chem.MolFromSmarts('[O-][C]([O-])=O')
        bicarbonate_pattern = Chem.MolFromSmarts('[OH][C]([O-])=O')
        carbonic_acid_pattern = Chem.MolFromSmarts('[OH][C]([OH])=O')
        cyanide_pattern = Chem.MolFromSmarts('[#6]#[#7]')
        
        # Check for carbon-carbon bonds
        has_cc_bond = mol.HasSubstructMatch(Chem.MolFromSmarts('[#6]-[#6]'))
        has_cc_double = mol.HasSubstructMatch(Chem.MolFromSmarts('[#6]=[#6]'))
        has_cc_triple = mol.HasSubstructMatch(Chem.MolFromSmarts('[#6]#[#6]'))
        has_cc_aromatic = mol.HasSubstructMatch(Chem.MolFromSmarts('[#6]:[#6]'))
        has_carbon_carbon = has_cc_bond or has_cc_double or has_cc_triple or has_cc_aromatic
        
        # If no carbon-carbon bonds, check if it's just an inorganic carbon compound
        if not has_carbon_carbon:
            is_co2 = mol.HasSubstructMatch(co2_pattern) if co2_pattern else False
            is_co = mol.HasSubstructMatch(co_pattern) if co_pattern else False
            is_carbonate = mol.HasSubstructMatch(carbonate_pattern) if carbonate_pattern else False
            is_bicarbonate = mol.HasSubstructMatch(bicarbonate_pattern) if bicarbonate_pattern else False
            is_carbonic = mol.HasSubstructMatch(carbonic_acid_pattern) if carbonic_acid_pattern else False
            is_cyanide = mol.HasSubstructMatch(cyanide_pattern) if cyanide_pattern else False
            
            if is_co2 or is_co or is_carbonate or is_bicarbonate or is_carbonic or is_cyanide:
                return False
        return True
    except Exception:
        return False

def generate_cpd_id(index, suffix):
    """Generate compound ID in format M{7-digit number}{suffix}"""
    return f"M{index+1:07d}{suffix}"

print("\n✓ Helper functions defined")


PARAMETERS
Compound file: /Users/bowen/Desktop/mcid/database/kegg/kegg_download.csv
Lipid file: /Users/bowen/Desktop/mcid/database/lipid/lipid_maps.csv
Datasource: KEGG
Rxn level: 0 Reaction
Compound ID index: K0
Output file: /Users/bowen/Desktop/mcid/database/kegg/kegg_filtered.parquet

✓ Helper functions defined


Data loading and validation

In [2]:
# ============================================================
# DATA LOADING AND VALIDATION
# ============================================================
print("Loading input files...")

# Load compounds (auto-detect CSV or Parquet)
compounds_df = detect_and_load_file(COMPOUND_FILE_PATH)
print(f"\nCompounds: {compounds_df.shape[0]} rows x {compounds_df.shape[1]} columns")

# Load lipids
lipids_df = detect_and_load_file(LIPID_FILE_PATH)
print(f"Lipids: {lipids_df.shape[0]} rows x {lipids_df.shape[1]} columns")

# Validate and standardize smiles column
if 'smiles' not in compounds_df.columns:
    raise ValueError("Compounds file must have a 'smiles' column!")

smiles_col_lipid = next((col for col in lipids_df.columns if col.lower() == 'smiles'), None)
if smiles_col_lipid is None:
    raise ValueError("Lipids file must have a 'smiles' column!")
if smiles_col_lipid != 'smiles':
    lipids_df = lipids_df.rename(columns={smiles_col_lipid: 'smiles'})

# Check for duplicate columns
for name, df in [('compounds', compounds_df), ('lipids', lipids_df)]:
    dup_cols = df.columns[df.columns.duplicated()].tolist()
    if dup_cols:
        print(f"WARNING: Duplicate columns in {name}: {dup_cols}")

print("\n✓ Data validation complete")


Loading input files...
  Detected CSV format: /Users/bowen/Desktop/mcid/database/kegg/kegg_download.csv

Compounds: 19571 rows x 8 columns
  Detected CSV format: /Users/bowen/Desktop/mcid/database/lipid/lipid_maps.csv
Lipids: 49671 rows x 21 columns

✓ Data validation complete


# Step 1: Get InChIKey main block and exact mass from SMILES

In [3]:
# ============================================================
# Step 1: Get InChIKey main block and exact mass from SMILES
# ============================================================
print("# Step 1: Get InChIKey main block and exact mass from SMILES")
print("=" * 60)

# Process compounds
compounds_df['inchikey_main_block'] = compounds_df['smiles'].apply(get_inchikey_main_block)
compounds_df['exact_mass'] = compounds_df['smiles'].apply(get_exact_mass)
print(f"Compounds - Valid InChIKey: {compounds_df['inchikey_main_block'].notna().sum()}/{len(compounds_df)}")
print(f"Compounds - Valid exact mass: {compounds_df['exact_mass'].notna().sum()}/{len(compounds_df)}")

# Process lipids
lipids_df['inchikey_main_block'] = lipids_df['smiles'].apply(get_inchikey_main_block)
print(f"Lipids - Valid InChIKey: {lipids_df['inchikey_main_block'].notna().sum()}/{len(lipids_df)}")

print("\n✓ Step 1 complete")

# Step 1: Get InChIKey main block and exact mass from SMILES


[22:44:48] Invalid InChI prefix in generating InChI Key
[22:44:48] Invalid InChI prefix in generating InChI Key
[22:44:48] Invalid InChI prefix in generating InChI Key
[22:44:48] Invalid InChI prefix in generating InChI Key
[22:44:48] Invalid InChI prefix in generating InChI Key
[22:44:48] Invalid InChI prefix in generating InChI Key
[22:44:48] Invalid InChI prefix in generating InChI Key
[22:44:48] Invalid InChI prefix in generating InChI Key
[22:44:48] Invalid InChI prefix in generating InChI Key
[22:44:48] Invalid InChI prefix in generating InChI Key
[22:44:48] WARNING: not removing hydrogen atom without neighbors
[22:44:48] Invalid InChI prefix in generating InChI Key
[22:44:48] Invalid InChI prefix in generating InChI Key
[22:44:48] Invalid InChI prefix in generating InChI Key
[22:44:48] Invalid InChI prefix in generating InChI Key
[22:44:48] Invalid InChI prefix in generating InChI Key
[22:44:48] Invalid InChI prefix in generating InChI Key
[22:44:48] Invalid InChI prefix in gene

Compounds - Valid InChIKey: 17160/19571
Compounds - Valid exact mass: 18774/19571


[22:45:02] SMILES Parse Error: syntax error while parsing: C([C@H](C)CO)C{-}/C=C(\C{+n}C/C=C(/CC/C=C(\C)/CC/C=C(\C)/C)\C)/C
[22:45:02] SMILES Parse Error: check for mistakes around position 15:
[22:45:02] C([C@H](C)CO)C{-}/C=C(\C{+n}C/C=C(/CC/C=C
[22:45:02] ~~~~~~~~~~~~~~^
[22:45:02] SMILES Parse Error: Failed parsing SMILES 'C([C@H](C)CO)C{-}/C=C(\C{+n}C/C=C(/CC/C=C(\C)/CC/C=C(\C)/C)\C)/C' for input: 'C([C@H](C)CO)C{-}/C=C(\C{+n}C/C=C(/CC/C=C(\C)/CC/C=C(\C)/C)\C)/C'
[22:45:02] SMILES Parse Error: syntax error while parsing: C([C@H](C)C(=O)O)C{-}/C=C(\C{+n}C/C=C(/CC/C=C(\C)/CC/C=C(\C)/C)\C)/C
[22:45:02] SMILES Parse Error: check for mistakes around position 19:
[22:45:02] C([C@H](C)C(=O)O)C{-}/C=C(\C{+n}C/C=C(/CC
[22:45:02] ~~~~~~~~~~~~~~~~~~^
[22:45:02] SMILES Parse Error: Failed parsing SMILES 'C([C@H](C)C(=O)O)C{-}/C=C(\C{+n}C/C=C(/CC/C=C(\C)/CC/C=C(\C)/C)\C)/C' for input: 'C([C@H](C)C(=O)O)C{-}/C=C(\C{+n}C/C=C(/CC/C=C(\C)/CC/C=C(\C)/C)\C)/C'
[22:45:02] SMILES Parse Error: syntax er

Lipids - Valid InChIKey: 49634/49671

✓ Step 1 complete


# Step 2: Remove lipids from compounds

In [4]:
# ============================================================
# Step 2: Remove lipids from compounds by comparing InChIKey main block
# ============================================================
print("# Step 2: Remove lipids from compounds by comparing InChIKey main block")
print("=" * 60)

lipid_inchikey_set = set(lipids_df['inchikey_main_block'].dropna().unique())
print(f"Unique lipid InChIKey main blocks: {len(lipid_inchikey_set)}")

initial_count = len(compounds_df)
is_lipid = compounds_df['inchikey_main_block'].isin(lipid_inchikey_set)
lipid_compounds = compounds_df[is_lipid]

if len(lipid_compounds) > 0:
    print(f"\nLipid compounds removed ({len(lipid_compounds)}):")
    for _, row in lipid_compounds.iterrows():
        print(f"  - {row.get('compound_id', 'N/A')}: {row.get('name', 'N/A')}")

compounds_df = compounds_df[~is_lipid].copy()
print(f"\nCompounds: {initial_count} → {len(compounds_df)} (removed {initial_count - len(compounds_df)})")
print("\n✓ Step 2 complete")

# Step 2: Remove lipids from compounds by comparing InChIKey main block
Unique lipid InChIKey main blocks: 43491

Lipid compounds removed (3064):
  - C00022: Pyruvate
  - C00024: Acetyl-CoA
  - C00033: Acetate
  - C00036: Oxaloacetate
  - C00042: Succinate
  - C00043: UDP-N-acetyl-alpha-D-glucosamine
  - C00065: L-Serine
  - C00077: L-Ornithine
  - C00083: Malonyl-CoA
  - C00091: Succinyl-CoA
  - C00100: Propanoyl-CoA
  - C00109: 2-Oxobutanoate
  - C00110: Dolichyl phosphate
  - C00123: L-Leucine
  - C00129: Isopentenyl diphosphate
  - C00136: Butanoyl-CoA
  - C00141: 3-Methyl-2-oxobutanoic acid
  - C00154: Palmitoyl-CoA
  - C00163: Propanoate
  - C00164: Acetoacetate
  - C00183: L-Valine
  - C00187: Cholesterol
  - C00203: UDP-N-acetyl-D-galactosamine
  - C00204: 2-Dehydro-3-deoxy-D-gluconate
  - C00207: Acetone
  - C00219: Arachidonate
  - C00223: p-Coumaroyl-CoA
  - C00232: Succinate semialdehyde
  - C00235: Dimethylallyl diphosphate
  - C00246: Butanoic acid
  - C00248: Lipoamide
 

# Step 3: Remove rows where InChIKey main block is null

In [5]:
# ============================================================
# Step 3: Remove rows where InChIKey main block is null
# ============================================================
print("# Step 3: Remove rows where InChIKey main block is null")
print("=" * 60)

initial_count = len(compounds_df)
null_inchikey_mask = compounds_df['inchikey_main_block'].isna() | (compounds_df['inchikey_main_block'] == '')
null_inchikey_compounds = compounds_df[null_inchikey_mask]

if len(null_inchikey_compounds) > 0:
    print(f"Compounds with null InChIKey main block removed ({len(null_inchikey_compounds)}):")
    for _, row in null_inchikey_compounds.iterrows():
        smiles_str = row['smiles'] if pd.notna(row['smiles']) else "N/A"
        print(f"  - {row.get('compound_id', 'N/A')}: {row.get('name', 'N/A')} (SMILES: {smiles_str})")

compounds_df = compounds_df[~null_inchikey_mask].copy()
print(f"\nCompounds: {initial_count} → {len(compounds_df)} (removed {initial_count - len(compounds_df)})")
print("\n✓ Step 3 complete")

# Step 3: Remove rows where InChIKey main block is null
Compounds with null InChIKey main block removed (2411):
  - C00012: Peptide (SMILES: *C(N)C(=O)NC(*)C(=O)O)
  - C00017: Protein (SMILES: *[C@H](N)C(=O)N[C@@H](*)C(=O)O)
  - C00028: Acceptor (SMILES: N/A)
  - C00030: Reduced acceptor (SMILES: N/A)
  - C00039: DNA (SMILES: *[C@H]1C[C@H](OP(=O)(O)OC[C@H]2O[C@@H](*)C[C@@H]2OP(=O)(O)OC[C@H]2O[C@@H](*)C[C@@H]2O)[C@@H](CO)O1)
  - C00040: Acyl-CoA (SMILES: *C(=O)SCCNC(=O)CCNC(=O)[C@H](O)C(C)(C)COP(=O)(O)OP(=O)(O)OC[C@H]1O[C@@H](n2cnc3c(N)ncnc32)[C@H](O)[C@@H]1OP(=O)(O)O)
  - C00045: Amino acid (SMILES: *C(N)C(=O)O)
  - C00046: RNA (SMILES: *[C@@H]1O[C@H](COP(=O)(O)O[C@H]2[C@@H](O)[C@H](*)O[C@@H]2COP(=O)(O)O[C@H]2[C@@H](O)[C@H](*)O[C@@H]2COP(=O)(O)O)[C@@H](O)[C@H]1O)
  - C00050: Metal (SMILES: N/A)
  - C00060: Carboxylate (SMILES: *C(=O)[O-])
  - C00066: tRNA (SMILES: *[C@@H]1O[C@H](COP(=O)(O)O[C@H]2[C@@H](O)[C@H](*)O[C@@H]2COP(=O)(O)O[C@H]2[C@@H](O)[C@H](*)O[C@@H]2CO)[C@@H](O)[C@H]1O)
  -

# Step 4: Remove rows with exact mass < 30

In [6]:
# ============================================================
# Step 4: Remove rows where exact mass is smaller than 30
# ============================================================
print("# Step 4: Remove rows where exact mass is smaller than 30")
print("=" * 60)

initial_count = len(compounds_df)
low_mass_mask = compounds_df['exact_mass'] < 30
low_mass_compounds = compounds_df[low_mass_mask]

if len(low_mass_compounds) > 0:
    print(f"Compounds with exact mass < 30 removed ({len(low_mass_compounds)}):")
    for _, row in low_mass_compounds.iterrows():
        print(f"  - {row.get('compound_id', 'N/A')}: {row.get('name', 'N/A')} (mass: {row['exact_mass']:.4f})")

compounds_df = compounds_df[~low_mass_mask].copy()
print(f"\nCompounds: {initial_count} → {len(compounds_df)} (removed {initial_count - len(compounds_df)})")
print("\n✓ Step 4 complete")

# Step 4: Remove rows where exact mass is smaller than 30
Compounds with exact mass < 30 removed (22):
  - C00001: H2O (mass: 18.0106)
  - C00014: Ammonia (mass: 17.0265)
  - C00080: H+ (mass: 1.0073)
  - C00177: Cyanide ion (mass: 26.0036)
  - C00237: CO (mass: 27.9949)
  - C00282: Hydrogen (mass: 2.0157)
  - C00305: Magnesium cation (mass: 23.9839)
  - C00533: Nitric oxide (mass: 29.9980)
  - C00697: Nitrogen (mass: 28.0061)
  - C00742: Fluoride (mass: 18.9990)
  - C01326: Hydrogen cyanide (mass: 27.0109)
  - C01328: HO- (mass: 17.0033)
  - C01330: Sodium cation (mass: 22.9892)
  - C01342: NH4+ (mass: 18.0338)
  - C01438: Methane (mass: 16.0313)
  - C01548: Acetylene (mass: 26.0157)
  - C06547: Ethylene (mass: 28.0313)
  - C15473: Lithium (mass: 8.0238)
  - C16460: Beryllium (mass: 11.0278)
  - C16487: Hydrofluoric acid (mass: 20.0062)
  - C16844: Hydroxyl radical (mass: 17.0027)
  - C19503: Polyethylene (mass: 28.0313)

Compounds: 14096 → 14074 (removed 22)

✓ Step 4 complete


# Step 5: Remove inorganic compounds (based on SMARTS)

In [7]:
# ============================================================
# Step 5: Remove inorganic compounds (based on SMARTS)
# ============================================================
print("# Step 5: Remove inorganic compounds (based on SMARTS)")
print("=" * 60)

initial_count = len(compounds_df)
compounds_df['is_organic'] = compounds_df['smiles'].apply(is_organic)
inorganic_compounds = compounds_df[~compounds_df['is_organic']]

if len(inorganic_compounds) > 0:
    print(f"Inorganic compounds removed ({len(inorganic_compounds)}):")
    for _, row in inorganic_compounds.iterrows():
        print(f"  - {row.get('compound_id', 'N/A')}: {row.get('name', 'N/A')} (SMILES: {row['smiles']})")

compounds_df = compounds_df[compounds_df['is_organic']].copy()
compounds_df = compounds_df.drop(columns=['is_organic'])
print(f"\nCompounds: {initial_count} → {len(compounds_df)} (removed {initial_count - len(compounds_df)})")
print("\n✓ Step 5 complete")


# Step 5: Remove inorganic compounds (based on SMARTS)
Inorganic compounds removed (183):
  - C00007: Oxygen (SMILES: O=O)
  - C00009: Orthophosphate (SMILES: O=P(O)(O)O)
  - C00011: CO2 (SMILES: O=C=O)
  - C00013: Diphosphate (SMILES: O=P(O)(O)OP(=O)(O)O)
  - C00023: Iron (SMILES: [Fe])
  - C00027: Hydrogen peroxide (SMILES: OO)
  - C00034: Manganese (SMILES: [Mn])
  - C00038: Zinc cation (SMILES: [Zn+2])
  - C00059: Sulfate (SMILES: O=S(=O)(O)O)
  - C00070: Copper (SMILES: [Cu])
  - C00076: Calcium cation (SMILES: [Ca+2])
  - C00087: Sulfur (SMILES: [S])
  - C00088: Nitrite (SMILES: O=NO)
  - C00094: Sulfite (SMILES: O=S(O)O)
  - C00150: Molybdenum (SMILES: [Mo])
  - C00175: Cobalt ion (SMILES: [Co+2])
  - C00192: Hydroxylamine (SMILES: NO)
  - C00238: Potassium cation (SMILES: [K+])
  - C00244: Nitrate (SMILES: O=[N+]([O-])O)
  - C00283: Hydrogen sulfide (SMILES: S)
  - C00288: HCO3- (SMILES: O=C([O-])O)
  - C00291: Nickel (SMILES: [Ni])
  - C00320: Thiosulfate (SMILES: O=S(=O)(O)[S

# Final Processing: Add metadata, generate IDs, save to Parquet

In [9]:
# ============================================================
# Final Processing: Add metadata, generate compound IDs, and save
# ============================================================
print("# Final Processing: Add metadata, generate compound IDs, and save")
print("=" * 60)

# Reset index and generate cpd_id
compounds_df = compounds_df.reset_index(drop=True)
compounds_df['cpd_id'] = [generate_cpd_id(i, CPD_ID_INDEX) for i in range(len(compounds_df))]

# Add metadata columns
compounds_df['datasource'] = DATASOURCE
compounds_df['rxn_level'] = RXN_LEVEL

# Reorder columns - cpd_id first
output_columns = ['cpd_id', 'name', 'exact_mass', 'smiles', 'inchikey_main_block', 'datasource', 'rxn_level', ]
original_cols = [col for col in compounds_df.columns if col not in output_columns]
output_columns.extend(original_cols)
compounds_df = compounds_df[[col for col in output_columns if col in compounds_df.columns]]

# Save to Parquet
compounds_df.to_parquet(OUTPUT_FILE_PATH, index=False)
parquet_size = os.path.getsize(OUTPUT_FILE_PATH)

# ============================================================
# SUMMARY
# ============================================================
print("\n" + "=" * 60)
print("PROCESSING SUMMARY")
print("=" * 60)
print(f"\nFinal output: {len(compounds_df)} compounds")
print(f"Compound ID range: {compounds_df['cpd_id'].iloc[0]} to {compounds_df['cpd_id'].iloc[-1]}")
print(f"Output file: {OUTPUT_FILE_PATH} ({parquet_size/1024:.1f} KB)")

print("\n" + "-" * 60)
print("First 5 rows:")
print(compounds_df[['cpd_id', 'compound_id', 'name', 'exact_mass']].head().to_string(index=False))

# Final Processing: Add metadata, generate compound IDs, and save

PROCESSING SUMMARY

Final output: 13891 compounds
Compound ID range: M0000001K0 to M0013891K0
Output file: /Users/bowen/Desktop/mcid/database/kegg/kegg_filtered.parquet (1529.5 KB)

------------------------------------------------------------
First 5 rows:
    cpd_id compound_id  name  exact_mass
M0000001K0      C00002   ATP  506.995745
M0000002K0      C00003  NAD+  664.116398
M0000003K0      C00004  NADH  665.124772
M0000004K0      C00005 NADPH  745.091102
M0000005K0      C00006 NADP+  744.082729
